<a href="https://colab.research.google.com/github/Imppel-9704/scrape_rottentomatoes_top300/blob/main/selenium_rottentomatoes_top300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 16.3 MB/s eta 0:00:00


In [2]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 snapd squashfs-tools systemd-hwe-hwdb udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
0 upgraded, 9 newly installed, 0 to remove and 20 not upgraded.
Need to get 30.1 MB of archives.
After this operation, 123 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.4 [598 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 squashfs-tools amd64 1:4.5-3build1 [159 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-up

In [17]:
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

rank = []
movie = []
link = []

def call_selenium():
  options = webdriver.ChromeOptions()
  options.add_argument("--headless")
  options.add_argument("--no-sandbox")
  options.add_argument("--disable-dev-shm-usage")
  driver = webdriver.Chrome(options=options)
  return driver

def scrape_data():
  driver = call_selenium()
  url = "https://editorial.rottentomatoes.com/guide/best-movies-of-all-time/"
  driver.get(url)

  # table = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.aligncenter")))
  tables = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table.aligncenter")))

  for table in tables:

    rows = table.find_elements(By.TAG_NAME, "tr")
    for r in rows:
      row_data = r.find_element(By.CSS_SELECTOR, "td[style*='width: 10%; height: 23px; text-align: center;']")
      rank.append(row_data.text.strip().replace(".", ""))
      title_link = r.find_element(By.CSS_SELECTOR, "a.title")
      movie.append(title_link.text.strip())
      link.append(title_link.get_attribute("href"))

  driver.quit()

  data = {
        r : {"title": m, "link": l} for r, m, l in zip(rank, movie, link)
    }
  return data

def extract_from_data(data):
  driver = call_selenium()
  for key, item in data.items():
    if item['link']: # Check if link is not null
      driver.get(item['link'])
      script = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "script#media-scorecard-json")))
      script_text = script.get_attribute("text")
      data_dict = json.loads(script_text.strip())
      # Get audience score and description
      audience_score = data_dict.get("audienceScore", {})
      item["description"] = data_dict.get("description")
      item["audience_like"] = audience_score.get("likedCount")
      item["audience_notlike"] = audience_score.get("notLikedCount")
      item["total_audience_review"] = audience_score.get("reviewCount")
      item["audience_score"] = audience_score.get("score")
      item["audience_sentiment"] = audience_score.get("sentiment")
      item["audience_score_percent"] = audience_score.get("scorePercent")
      # Get critics score
      critics_score = data_dict.get("criticsScore", {})
      item["critic_like"] = critics_score.get("likedCount")
      item["critic_notlike"] = critics_score.get("notLikedCount")
      item["total_critic_review"] = critics_score.get("reviewCount")
      item["critic_score"] = critics_score.get("score")
      item["critic_sentiment"] = critics_score.get("sentiment")
      item["critic_score_percent"] = critics_score.get("scorePercent")

      crew = driver.find_element(By.CSS_SELECTOR, "div[data-modulecastcrewmanager= 'container']")
      if crew:
        director = driver.find_element(By.CSS_SELECTOR, "p.name")
        item["director"] = director.text.strip() if director else ""

      information = driver.find_element(By.CSS_SELECTOR, "div.media-hero-wrap")
      metadata = information.find_elements(By.CSS_SELECTOR, "rt-text[slot='metadataProp'")
      item["metadata"] = [md.text.strip().replace(",", "") for md in metadata]
      all_genre = information.find_elements(By.CSS_SELECTOR, "rt-text[slot='metadataGenre'")
      item["genre"] = [g.text.strip().replace("/", "") for g in all_genre]

  driver.quit()
  return data

def transform_data(data):
  for key, item in data.items():
    if "metadata" in item:
      item["rate"], item["released date"], item["duration"] = ([""] + item["metadata"])[:3]
      item["genre"] = ", ".join(item["genre"])
      item.pop("metadata")

  df = pd.DataFrame.from_dict(data, orient= "index").reset_index()
  df.rename(columns= {"index": "no"}, inplace=True)

  return df

def save_to_csv(df):
  df.to_csv("selenium_top300_rottentomatoes.csv", index=False)

def main():
  data = scrape_data()
  extracted_data = extract_from_data(data)
  dataframe = transform_data(extracted_data)
  save_to_csv(dataframe)

if __name__ == "__main__":
  main()